In [64]:
import pandas as pd
import numpy as np
import random
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import curve_fit
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime, timedelta
import numpy as np
from scipy.optimize import minimize, basinhopping


# 100銘柄のISINを生成
isin_list = ['ISIN{:03}'.format(i) for i in range(1, 5001)]

# データを格納するための空のリストを作成
data = []

# 同じISINで同じクーポン率、時価、信用格付けを生成
coupon_rates = {isin: random.uniform(0.01, 0.10) for isin in isin_list}
market_prices = {isin: random.uniform(80, 120) for isin in isin_list}
credit_ratings = {isin: random.choice([5, 4, 3, 2, 1, 0]) for isin in isin_list}

# 業種のリストを作成
industries = ['Technology', 'Finance', 'Healthcare', 'Energy', 'Consumer Goods']

# 月ごとのデータを生成
start_date = datetime(2023, 1, 1)
end_date = datetime(2023, 12, 31)
current_date = start_date

while current_date <= end_date:
    for i, isin in enumerate(isin_list):
        oas = random.uniform(0.1, 2.0)  # OASをランダムに生成
        t_spread = random.uniform(0.05, 1.0) * i / 100   # T-Spreadをランダムに生成
        z_spread = random.uniform(0.2, 3.0)  # Z-Spreadをランダムに生成
        remaining_years = random.uniform(0.1, 50)  # 残存年数をランダムに設定
        maturity_date = current_date.replace(year=current_date.year + 1)  # 償還日を1年後に設定
        payment_frequency = random.choice([1, 2, 4])  # 利払い回数をランダムに選択
        coupon_rate = coupon_rates[isin]  # 同じISINのクーポン率を取得
        market_price = market_prices[isin]  # 同じISINの時価を取得
        credit_rating = credit_ratings[isin]  # 同じISINの信用格付けを取得
        industry = random.choice(industries)  # ランダムな業種を選択
        flag_SDGs = random.choice([True, False])

        # データをリストに追加
        data.append([current_date, isin, oas, t_spread, z_spread, remaining_years, maturity_date, payment_frequency, coupon_rate, market_price, credit_rating, industry, flag_SDGs])

    # 月を進める
    if current_date.month == 12:
        current_date = current_date.replace(year=current_date.year + 1, month=1, day=1)
    else:
        current_date = current_date.replace(month=current_date.month + 1, day=1)

# データフレームを作成
columns = ['日付', 'ISIN', 'OAS', 'T-Spread', 'Z-Spread', '残存年数', '償還日', '利払い回数', 'クーポン率', '時価', '信用格付け', '業種', 'merged_green_flag']
df = pd.DataFrame(data, columns=columns)
df

,日付,ISIN,OAS,T-Spread,Z-Spread,残存年数,償還日,利払い回数,クーポン率,時価,信用格付け,業種,merged_green_flag
0,2023-01-01,ISIN001,0.193857,0.000000,1.738140,26.056285,2024-01-01,1,0.084907,102.212496,2,Consumer Goods,True
1,2023-01-01,ISIN002,0.429830,0.003659,2.689048,46.655404,2024-01-01,2,0.081814,104.025152,4,Finance,False
2,2023-01-01,ISIN003,1.083203,0.015246,2.610036,34.036414,2024-01-01,4,0.091181,116.875081,0,Consumer Goods,False
3,2023-01-01,ISIN004,1.385097,0.002934,2.043499,22.082755,2024-01-01,1,0.039919,82.900639,2,Technology,False
4,2023-01-01,ISIN005,1.018491,0.024689,2.745670,40.610696,2024-01-01,2,0.037809,114.183278,2,Healthcare,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,2023-12-01,ISIN4996,0.453430,7.300588,2.357653,34.701413,2024-12-01,4,0.081302,118.085144,1,Technology,False
59996,2023-12-01,ISIN4997,0.292044,13.279362,2.882587,4.232333,2024-12-01,1,0.087387,117.043084,5,Finance,True
59997,2023-12-01,ISIN4998,0.877654,41.187529,2.817387,2.564015,2024-12-01,4,0.050446,91.661924,4,Consumer Goods,False
59998,2023-12-01,ISIN4999,0.428110,39.279042,0.801414,39.064355,2024-12-01,4,0.075838,85.537160,4,Consumer Goods,True


In [53]:
# Nelson-Siegelモデルのパラメータ推計の計算最適化を含む関数
# Nelson-Siegelモデルの関数定義
def nelson_siegel_model(t, beta0, beta1, beta2, tau):
    return beta0 + (beta1 + beta2) * ((1 - np.exp(-t/tau)) / (t/tau)) - beta2 * np.exp(-t/tau)

def plot_data_and_fit_ns_model_optimized(df, metric, date_column, sector_column, rating_column, maturity_column, rating_split, maturity_step, min_num_of_data, flag_column):
    """ データを分割し、Nelson-Siegelモデルに基づいて推計し、プロットする関数

    Args:
        df : pd.DataFrame
            データフレーム
        metric : str
            目的変数(y軸)
        date_column : str
            日付データカラム
        sector_column : str
            セクターカラム
        rating_column : str
            信用格付けカラム
        maturity_column : str
            残存年数カラム
        rating_split : int
            信用格付けの分割数
        maturity_step : int
            残存年数の刻み幅
        min_num_of_data : int
            パラメータ推計に必要な最小データ点数
        flag_column : str
            フラグカラム
    """
    # 日付データでグループ分け
    for date, date_group in df.groupby(date_column):
        formatted_date = date.strftime('%Y-%m-%d')
        # PDFに複数ページを作成
        with PdfPages(f'{formatted_date}_{metric}_split.pdf') as pdf:
            # セクターごとに処理
            for sector in date_group[sector_column].unique():
                sector_df = date_group[date_group[sector_column] == sector]

                # 信用格付けを指定された数で分割
                sector_df['RatingGroup'] = pd.qcut(sector_df[rating_column], rating_split, labels=False, duplicates='drop')

                # 各信用格付けグループごとにプロット
                fig, axes = plt.subplots(rating_split, 1, figsize = (10, 6 * rating_split))
                fig.suptitle(f'{sector} - {date}')

                for i in range(rating_split):
                    rating_group_df = sector_df[sector_df['RatingGroup'] == i]
                    # 残存年数ごとの平均値を計算
                    avg_metric = rating_group_df.groupby((rating_group_df[maturity_column] // maturity_step).astype(int))[metric].mean()
                    print(avg_metric)
                    # Nelson-Siegelモデルのパラメータを推計
                    if len(avg_metric) >= min_num_of_data:
                        try:
                            maturity_values = np.array(avg_metric.index) * maturity_step
                            # 初期パラメータを設定してcurve_fitを呼び出し
                            popt, _ = curve_fit(nelson_siegel_model, maturity_values, avg_metric, p0=[0, 0, 0, 1], maxfev = 10000)

                            # 推計された曲線を描画
                            t_fit = np.linspace(0, maturity_values.max(), 100)
                            curve_fit = nelson_siegel_model(t_fit, *popt)
                            axes[i].plot(t_fit, curve_fit, label='Fitted Curve')

                        except RuntimeError:
                            # パラメータ推計に失敗した場合の処理
                            axes[i].text(0.5, 0.5, 'Fit failed', horizontalalignment = 'center', verticalalignment = 'center', transform = axes[i].transAxes)
                            continue

                    # 元のデータの散布図を描画（フラグに基づく色分け）
                    for flag_value in rating_group_df[flag_column].unique():
                        flag_df = rating_group_df[rating_group_df[flag_column] == flag_value]
                        color = 'red' if flag_value else 'blue'
                        axes[i].scatter(flag_df[maturity_column], flag_df[metric], color=color, label=f'Flag {flag_value}')

                    axes[i].set_title(f'Rating Group {i}')
                    axes[i].legend()

                plt.tight_layout()
                pdf.savefig()
                plt.close()


In [71]:
# Nelson-Siegelモデル関数
def nelson_siegel(t, beta0, beta1, beta2, tau):
    return beta0 + beta1 * ((1 - np.exp(-t/tau)) / (t/tau)) + beta2 * ((1 - np.exp(-t/tau)) / (t/tau) - np.exp(-t/tau))

# 誤差関数
def ns_error(params, t, observed_yields):
    beta0, beta1, beta2, tau = params
    predicted_yields = nelson_siegel(t, beta0, beta1, beta2, tau)
    return np.sum((observed_yields - predicted_yields)**2)

# Nelson-Siegelモデルのパラメータ推計の計算最適化を含む関数
def plot_data_and_fit_ns_model_optimized(df, metric, date_column, sector_column, rating_column, maturity_column, rating_split, maturity_step, min_num_of_data, flag_column):
    """ データを分割し、Nelson-Siegelモデルに基づいて推計し、プロットする関数

    Args:
        df : pd.DataFrame
            データフレーム
        metric : str
            目的変数(y軸)
        date_column : str
            日付データカラム
        sector_column : str
            セクターカラム
        rating_column : str
            信用格付けカラム
        maturity_column : str
            残存年数カラム
        rating_split : int
            信用格付けの分割数
        maturity_step : int
            残存年数の刻み幅
        min_num_of_data : int
            パラメータ推計に必要な最小データ点数
        flag_column : str
            フラグカラム
    """

    for date, date_group in df.groupby(date_column):
        formatted_date = date.strftime('%Y-%m-%d')
        with PdfPages(f'{formatted_date}_{metric}_split.pdf') as pdf:
            for sector in date_group[sector_column].unique():
                sector_df = date_group[date_group[sector_column] == sector]

                sector_df = sector_df.dropna(subset=[rating_column])
                if not np.issubdtype(sector_df[rating_column].dtype, np.number):
                    continue

                sector_df['RatingGroup'] = pd.qcut(sector_df[rating_column], rating_split, labels=False, duplicates='drop')

                fig, axes = plt.subplots(rating_split, 1, figsize=(10, 6 * rating_split))
                fig.suptitle(f'{sector} - {formatted_date}')

                for i in range(rating_split):
                    rating_group_df = sector_df[sector_df['RatingGroup'] == i]
                    avg_metric = rating_group_df.groupby((rating_group_df[maturity_column] // maturity_step).astype(int))[metric].mean()

                    # curve_fit結果の初期化
                    curve_fit_result = None

                    if len(avg_metric) >= min_num_of_data:
                        try:
                            maturity_values = np.array(avg_metric.index) * maturity_step
                            initial_params = [0.1, 0.1, 0.1, 1]
                            result = basinhopping(
                                                ns_error,
                                                initial_params,
                                                minimizer_kwargs = {"args": (maturity_values, avg_metric),
                                                                    "method": 'Nelder-Mead'}
                                                )
                            # result = minimize(ns_error,
                            #                   initial_params = [0.1, 0.1, 0.1, 1],
                            #                   args = (maturity_values, avg_metric),
                            #                   method = 'Nelder-Mead'
                            #                   )

                            # popt, _ = curve_fit(nelson_siegel_model, maturity_values, avg_metric, p0=[0.1, 0.1, 0.1, 1], maxfev = 20000)
                            t_fit = np.linspace(0, maturity_values.max(), 100)
                            curve_fit_result = nelson_siegel_model(t_fit, *result.x)
                        except RuntimeError:
                            axes[i].text(0.5, 0.5, 'Fit failed', horizontalalignment='center', verticalalignment='center', transform=axes[i].transAxes)
                            continue

                    if curve_fit_result is not None:
                        axes[i].plot(t_fit, curve_fit_result, label='Fitted Curve')

                    for flag_value in rating_group_df[flag_column].unique():
                        flag_df = rating_group_df[rating_group_df[flag_column] == flag_value]
                        color = 'red' if flag_value else 'blue'
                        axes[i].scatter(flag_df[maturity_column], flag_df[metric], color=color, label=f'Flag {flag_value}')

                    axes[i].set_title(f'Rating Group {i}')
                    axes[i].legend()

                plt.tight_layout()
                pdf.savefig()
                plt.close()

In [72]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from matplotlib.backends.backend_pdf import PdfPages

plot_data_and_fit_ns_model_optimized(df, 'T-Spread', '日付', '業種', '信用格付け', '残存年数', 4, 2, 20, "merged_green_flag")

C:\Users\user\AppData\Local\Temp\ipykernel_9764\745492582.py:3: RuntimeWarning: invalid value encountered in divide
  return beta0 + beta1 * ((1 - np.exp(-t/tau)) / (t/tau)) + beta2 * ((1 - np.exp(-t/tau)) / (t/tau) - np.exp(-t/tau))


2 残存年数
0     18.626960
1     17.293355
2     10.221542
3     11.278846
4     13.572456
5      9.052797
6     15.775096
7     13.601515
8      9.415841
9      9.388831
10    14.047154
11    11.587217
12    16.495835
13    17.572437
14    12.905330
15    12.084456
16    11.415590
17    12.100473
18    17.587477
19    14.950077
20    11.483143
21    14.869057
22    13.483290
23    21.986447
24    17.201744
Name: T-Spread, dtype: float64                     message: ['requested number of basinhopping iterations completed successfully']
                    success: True
                        fun: 199.16951676045616
                          x: [ 3.363e+05 -3.363e+05  5.219e+05  4.789e+06]
                        nit: 100
      minimization_failures: 1
                       nfev: 33274
 lowest_optimization_result:       message: Optimization terminated successfully.
                                   success: True
                                    status: 0
                             

C:\Users\user\AppData\Local\Temp\ipykernel_9764\3196615387.py:4: RuntimeWarning: invalid value encountered in divide
  return beta0 + (beta1 + beta2) * ((1 - np.exp(-t/tau)) / (t/tau)) - beta2 * np.exp(-t/tau)
C:\Users\user\AppData\Local\Temp\ipykernel_9764\745492582.py:3: RuntimeWarning: invalid value encountered in divide
  return beta0 + beta1 * ((1 - np.exp(-t/tau)) / (t/tau)) + beta2 * ((1 - np.exp(-t/tau)) / (t/tau) - np.exp(-t/tau))


2 残存年数
0     10.667473
1      8.971701
2      9.577124
3     17.639595
4     26.751542
5     14.397969
6     12.734937
7     17.774178
8     11.204489
9     12.138642
10    14.035032
11    10.521211
12    11.944048
13     7.634527
14     9.631508
15    13.750080
16    12.779622
17     5.612328
18    11.598944
19    11.471083
20    16.200205
21    18.468188
22     7.463808
23    13.740057
24    20.593465
Name: T-Spread, dtype: float64                     message: ['requested number of basinhopping iterations completed successfully']
                    success: True
                        fun: 461.3436829000898
                          x: [ 2.894e+04 -2.892e+04 -2.956e+04  7.986e+02]
                        nit: 100
      minimization_failures: 63
                       nfev: 60519
 lowest_optimization_result:       message: Optimization terminated successfully.
                                   success: True
                                    status: 0
                             

C:\Users\user\AppData\Local\Temp\ipykernel_9764\3196615387.py:4: RuntimeWarning: invalid value encountered in divide
  return beta0 + (beta1 + beta2) * ((1 - np.exp(-t/tau)) / (t/tau)) - beta2 * np.exp(-t/tau)
C:\Users\user\AppData\Local\Temp\ipykernel_9764\745492582.py:3: RuntimeWarning: invalid value encountered in divide
  return beta0 + beta1 * ((1 - np.exp(-t/tau)) / (t/tau)) + beta2 * ((1 - np.exp(-t/tau)) / (t/tau) - np.exp(-t/tau))


...

C:\Users\user\AppData\Local\Temp\ipykernel_9764\3196615387.py:4: RuntimeWarning: invalid value encountered in divide
  return beta0 + (beta1 + beta2) * ((1 - np.exp(-t/tau)) / (t/tau)) - beta2 * np.exp(-t/tau)
C:\Users\user\AppData\Local\Temp\ipykernel_9764\745492582.py:3: RuntimeWarning: invalid value encountered in divide
  return beta0 + beta1 * ((1 - np.exp(-t/tau)) / (t/tau)) + beta2 * ((1 - np.exp(-t/tau)) / (t/tau) - np.exp(-t/tau))


2 残存年数
0     10.541302
1     16.124052
2     15.287573
3      9.623458
4     11.878256
5     10.614291
6     13.498801
7     10.689342
8     16.698451
9     11.816042
10    10.454986
11    15.448432
12    11.025520
13    12.798304
14    14.379403
15    12.925505
16    11.327947
17    18.865200
18    15.215609
19    13.670636
20    14.368317
21    10.318587
22    13.812397
23    13.462135
24    12.509940
Name: T-Spread, dtype: float64                     message: ['requested number of basinhopping iterations completed successfully']
                    success: True
                        fun: 101.96427962812756
                          x: [ 1.445e+01  1.114e+01 -2.343e+01  2.547e+00]
                        nit: 100
      minimization_failures: 0
                       nfev: 24233
 lowest_optimization_result:       message: Optimization terminated successfully.
                                   success: True
                                    status: 0
                             

C:\Users\user\AppData\Local\Temp\ipykernel_9764\3196615387.py:4: RuntimeWarning: invalid value encountered in divide
  return beta0 + (beta1 + beta2) * ((1 - np.exp(-t/tau)) / (t/tau)) - beta2 * np.exp(-t/tau)
C:\Users\user\AppData\Local\Temp\ipykernel_9764\745492582.py:3: RuntimeWarning: invalid value encountered in divide
  return beta0 + beta1 * ((1 - np.exp(-t/tau)) / (t/tau)) + beta2 * ((1 - np.exp(-t/tau)) / (t/tau) - np.exp(-t/tau))
C:\Users\user\AppData\Local\Temp\ipykernel_9764\745492582.py:3: RuntimeWarning: overflow encountered in exp
  return beta0 + beta1 * ((1 - np.exp(-t/tau)) / (t/tau)) + beta2 * ((1 - np.exp(-t/tau)) / (t/tau) - np.exp(-t/tau))
C:\Users\user\AppData\Local\Temp\ipykernel_9764\745492582.py:3: RuntimeWarning: invalid value encountered in subtract
  return beta0 + beta1 * ((1 - np.exp(-t/tau)) / (t/tau)) + beta2 * ((1 - np.exp(-t/tau)) / (t/tau) - np.exp(-t/tau))


2 残存年数
0      8.080032
1     14.009223
2     13.745126
3     15.621125
4     14.000236
5     12.373328
6     13.277072
7     17.925540
8     12.800284
9     14.342423
10    11.432778
11    12.049780
12    13.344265
13     9.365051
14    15.555810
15    13.097458
16    20.138173
17     9.279358
18     8.710155
19    11.807853
20    14.775630
21    10.666160
22    11.320320
23    17.141768
24    18.122439
Name: T-Spread, dtype: float64                     message: ['requested number of basinhopping iterations completed successfully']
                    success: True
                        fun: 189.10245961842284
                          x: [ 1.305e+01 -2.298e+00  7.650e+00  1.590e+00]
                        nit: 100
      minimization_failures: 66
                       nfev: 63258
 lowest_optimization_result:       message: Optimization terminated successfully.
                                   success: True
                                    status: 0
                            

C:\Users\user\AppData\Local\Temp\ipykernel_9764\3196615387.py:4: RuntimeWarning: invalid value encountered in divide
  return beta0 + (beta1 + beta2) * ((1 - np.exp(-t/tau)) / (t/tau)) - beta2 * np.exp(-t/tau)
C:\Users\user\AppData\Local\Temp\ipykernel_9764\745492582.py:3: RuntimeWarning: invalid value encountered in divide
  return beta0 + beta1 * ((1 - np.exp(-t/tau)) / (t/tau)) + beta2 * ((1 - np.exp(-t/tau)) / (t/tau) - np.exp(-t/tau))
